**Data Warehouse and Data Mining**
* Group Members
* Kevoy Walters - 1403487
* Jevaun Harris - 1403164
* Sheneal McCourtie - 1501076
* Shanygne Downey - 1503760


**Description of Dataset**

This dataset includes information which will help predict customer behaviour in order to ensure satisfaction and minimize churn. Each row represents a customer, each column contains customer’s attributes described on the column Metadata. The raw data contains 7043 rows (customers) and 21 columns (features). The ‘Churn’ column is our target. Excluding the ‘customerID’ column, there are 3 numerical and 17 categorical columns. The data set includes information about:
* Customers who left within the last month – the column is called Churn
** Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents



**Background**

Flashpoint Communications is a telecommunications company which provides several services such as telephone and internet service, tv and movie streaming, tech support, device protection, online backup and online security. The company is concerned about the number of customers leaving their landline business for cable competitors. They need to understand who is leaving and why. This attrition or turnover of customers of a business or users of a service is called churn. Having a high churn rate is bad for the company as is reduces growth and profit and that is the opposite of what Flashpoint Communications want.   


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
data

In [ ]:
# output = (rows, columns)
data.shape 

**Target Variable - Churn**

Which features are numerical?
SeniorCitizen, Tenure, MonthlyCharges, TotalCharges

Continous - Tenure, MonthlyCharges, TotalCharges

Discrete - SeniorCitizen

Which features are categorical?
PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod, gender, Partner, Dependents

This dataset is a categorical dataset since most of its columns including the target varibles are categorical data

In [ ]:
# making a copy of the data and saving it in a variable 
data_temp = data.copy()

**Visualization on the Dataset**

In [ ]:
data_temp['Churn'].value_counts().plot.pie(autopct='%1.1f%%')

Pie chart showing the percentage of who churn which is 26.5% and those who stayed with the company which is 73.5%

In [ ]:
sns.factorplot(x="Contract", y="MonthlyCharges", hue="PaymentMethod", kind="point", data=data_temp)

Point chart showing the relationship between the type of contarct and the month charges also using the type of payment method as the key

In [ ]:
sns.pairplot(data_temp[['tenure','MonthlyCharges','TotalCharges','Churn','Contract','SeniorCitizen']], hue='Churn')

In [ ]:
# shorter the contract period, higher probability of churn? lets check crosstab

cr  = pd.crosstab(data_temp.Contract, data_temp.Churn)
cr.plot(kind='bar')
plt.title('Churn by Contract')
plt.ylabel("Number of chrun")
plt.show()

bar garph showing the number of customers churn against the type of contract

In [ ]:
data_temp.dtypes

**Preprocessing**

Here we can see that Total Charges is an object variable. Let's Change it to float

In [ ]:
tot_charges = data_temp["TotalCharges"]

In [ ]:
def convert_to_float(strin):
        try:
            return float(strin)
        except:
            return 0

In [ ]:
data_temp['TotalCharges']=data_temp['TotalCharges'].apply(convert_to_float)

In [ ]:
data_temp['TotalCharges']

In [ ]:
#describe the data
data_temp.describe()

In [ ]:
# Checking For NULL 
data_temp.isnull().sum()

In [ ]:
#Identifying the rows containing 0 value in Total Charges
zero_value_row = list(data_temp[data_temp['TotalCharges'] == 0].index)
print('0 Value Rows=',zero_value_row, "\ntotal =", len(zero_value_row))


In [ ]:
for zero_row in zero_value_row :
    print( data_temp['MonthlyCharges'][zero_row],data_temp['tenure'][zero_row],data_temp['TotalCharges'][zero_row])

So Tenure is 0 for the cstomers with TotalCharges as 0, that initially had a space.  might mean that they have not yet been with the company for a whole month, and therefore have not yet been charged any money. This explains why TotalCharges is missing, and also why none of them have churned yet.

 

In [ ]:
# Look at the shape before and after to be sure they were removed
print(data_temp.shape)
data_temp = data_temp.drop(data_temp.index[[488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]])
print(data_temp.shape)

**Let's Convert Our Target Variable 'Churn' for Yes or No to 1 or 0.**
* Using the logistic regression model, categories must be transformed into numbers first before we can apply the learning algorithm.


In [ ]:
#coverting Churn from Yes or No to 1 or 0 and saving it in a new column name class
data_temp['class'] = data_temp['Churn'].apply(lambda x : 1 if x == "Yes" else 0)

In [ ]:
count_no_churn = len(data_temp[data_temp['class']==0])
count_churn = len(data_temp[data_temp['class']==1])
pct_of_no_churn = count_no_churn/(count_no_churn+count_churn)
print("percentage of no churn is", pct_of_no_churn*100)
pct_of_churn = count_churn/(count_no_churn+count_churn)
print("percentage of churn", pct_of_churn*100)

**when churn is "Yes"**

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'MonthlyCharges'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'TotalCharges'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'tenure'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'), 'Contract'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'PaymentMethod'].value_counts(normalize = True)

**Most of the People that Left are the Ones who had Payment Method as Electronic Check so Let's Make a Seperate Variable for it so that The Model can Easily Predict our Target Variable.**

In [ ]:
data_temp['Is_Electronic_check'] = np.where(data_temp['PaymentMethod'] == 'Electronic check',1,0)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'PaperlessBilling'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'), 'Contract'].value_counts(normalize = True)

In [ ]:
data_temp['Is_Contract_Month'] = np.where(data_temp['PaymentMethod'] == 'Month-to-month',1,0)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'DeviceProtection'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'OnlineBackup'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'TechSupport'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'OnlineSecurity'].value_counts(normalize = True)

**We can See that People That Left the Company did'nt use Services Like Online Security , Device Protection , Tech Support and Online Backup quite often. Hence for Our Prediction these variables will not be of much Importance. We will Drop them in the End.**

In [ ]:
data_temp= pd.get_dummies(data_temp,columns=['Partner','Dependents',
       'PhoneService', 'MultipleLines','StreamingTV',
       'StreamingMovies','Contract','PaperlessBilling','InternetService'],drop_first=True)

We have Encoded the Categorical Variables with Numeric using get dummies Property which will make it easy for the Machine to Make Correct Prediction.

In [ ]:
data_temp.info()

**Transformation**
* Transforming the data to get the best outcome  from the algorithms

Now Let's Drop the variables that are not Important For us according to our Analysis.

In [ ]:
data_temp.drop(['StreamingTV_No internet service','StreamingMovies_No internet service'],axis=1,inplace=True)

In [ ]:
data_temp.drop('gender',axis=1,inplace=True)

In [ ]:
data_temp.drop('class',axis=1,inplace=True)

In [ ]:
data_temp.drop('customerID',axis=1,inplace=True)

In [ ]:
#data_temp.drop(['tenure','MonthlyCharges'],axis=1,inplace=True)

In [ ]:
data_temp.drop(['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','PaymentMethod'],axis=1,inplace=True)

In [ ]:
data_temp = pd.get_dummies(data_temp,columns=['Churn'],drop_first=True)

In [ ]:
data_temp.info()

**Data Mining Algorithm**

Description of algorithms
A total of three algorithms were used to analyze and produce evaluations for the telecommunication dataset. Each algorithm has its own description, application and advantages. There algorithms used were Decision trees, Logistic regression and K-Modes. 


**Let's start with Logistic Regression Model because we know Our Target Variable has a Binary Outcome.**
* Binary logistic regression requires the dependent variable to be binary.
* For a binary regression, the factor level 1 of the dependent variable should represent the desired outcome.
* Only the meaningful variables should be included.

 Logistic regression gives you a discrete outcome but linear regression gives a continuous outcome.

**Logistic Regression**
* Regression is a technique for determining the statistical relationship between two or more variables where a change in a dependent variable is associated with, and depends on, a change in one or more independent variables. There are multiple types of regressions available in data mining however logistic regression was fit for our dataset and its columns. There are several benefits of using regression analysis, two of which are:
1. It indicates the significant relationships between dependent variable and independent variable.
2. It indicates the strength of impact of multiple independent variables on a dependent variable.

Logistic regression is used to find the probability of event=Success and event=Failure. Logistic regression is used when the dependent variable is binary (0/ 1, True/ False, Yes/ No) in nature. 


This type of regression is widely used for classification problems. In general, according to analyticsvidhya.com (2018), there are few important points to note when using logistic regression as a data mining technique. They are:
* Logistic regression doesn’t require linear relationship between dependent and independent variables.  It can handle various types of relationships because it applies a non-linear log transformation to the predicted odds ratio
* To avoid overfitting and underfitting, we should include all significant variables. A good approach to ensure this practice is to use a step wise method to estimate the logistic regression
* It requires large sample sizes because maximum likelihood estimates are less powerful at low sample sizes than ordinary least square
* The independent variables should not be correlated with each other i.e. no multicollinearity.  However, we have the options to include interaction effects of categorical variables in the analysis and in the model.
* If the values of dependent variable is ordinal, then it is called as Ordinal logistic regression
* If dependent variable is multi class then it is known as Multinomial Logistic regression.


In [ ]:
X = data_temp.drop('Churn_Yes',axis=1).values.astype('float')
y = data_temp['Churn_Yes']

In [ ]:
# train test 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
# create model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
# train model
model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,classification_report

In [ ]:
print('Model Score:',model.score(X_train,y_train))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print ('confusion matrix for logistic regression \n ', cnf_matrix)

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

**The results from the confusion matrix are telling us that 1404 and 265 are the number of correct predictions. 151 and 290 are the number of incorrect predictions.**

Precision is the fraction of correctly identified examples of a class (ratio of true possitive to all positives)
Recall is the fraction of observation classified in that class that was correctly classified

In [ ]:
print ('accuracy for logistic regression  : {0:.2f}'.format(accuracy_score(y_test, model.predict(X_test))))
print ('precision for logistic regression : {0:.2f}'.format(precision_score(y_test, model.predict(X_test))))
print ('recall for logistic regression    : {0:.2f}'.format(recall_score(y_test, model.predict(X_test))))

**We have a classification rate of 80%, considered as good accuracy.**

In [ ]:
print(classification_report(y_test,model.predict(X_test)))

**Plotting Output of the Logistic Resggresion**

**ROC Curve**
Receiver Operating Characteristic(ROC) curve is a plot of the true positive rate against the false positive rate. It shows the tradeoff between sensitivity and specificity.

In [ ]:
y_pred_prob = model.predict(X_test)
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_prob)
auc = metrics.roc_auc_score(y_test, y_pred_prob)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.show()

**AUC score for the case is 0.69 AUC score 1 represents perfect classifier, and 0.5 represents a worthless classifier.**

**Clustering Algorithm**
* Clustering (K Modes) 

K-Modes (Clustering)
Utilizing clustering techniques in machine learning forms a crucial part in data mining or analysis because diving the entire dataset in similar subgroups helps in gaining a lot of insight from the data. This unsupervised algorithm uses modes instead of the usual means (K-Means) to form clusters of categorical data. Firstly, mode can be defined as the value that occurs most frequently in a given set of data. K-Modes helps to provide insights on specific subgroups in a dataset, allowing data scientists to evaluate data or client structures and provide recommendations and counter measures in organizations.  

In [ ]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)  

In [ ]:
from kmodes.kmodes import KModes 

In [ ]:
data_temp2 = data_temp.copy()

In [ ]:
data_temp=data_temp.drop(['TotalCharges', 'tenure', 'MonthlyCharges'], axis=1)

In [ ]:

# random categorical data

km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(data_temp)



In [ ]:
km.cluster_centroids_

In [ ]:
clusters

In [ ]:
data_temp['Cluster_Group']= clusters

In [ ]:
data

In [ ]:
data_temp.columns

In [ ]:
data_temp['Cluster_Group'].value_counts().plot(kind='bar',title='Distribution of Customers across groups')
plt.xlabel("Clusters")
plt.ylabel("Number of Clusters")

In [ ]:
ct  = pd.crosstab(data_temp['Cluster_Group'],data_temp.Churn_Yes)
ct.plot(kind='bar')
plt.title('Churn rate of each Cluster Group')
plt.show()

In [ ]:
data_temp

In [ ]:
pd.DataFrame({
    'clusters':clusters,
    'SeniorCitizen': data_temp['SeniorCitizen'],
    'Churn_Yes': data_temp['Churn_Yes']
}).plot(kind='scatter', x='SeniorCitizen', y='Churn_Yes',c="clusters")
plt.xlabel('SeniorCitizen')
plt.figure(figsize=(13,13))
plt.show()

In [ ]:
cgroup = data_temp.groupby('Cluster_Group')

In [ ]:
cgroup.apply(lambda x : x.mode())

In [ ]:
sns.pairplot(data_temp[['Is_Contract_Month','Contract_One year','Contract_Two year','Cluster_Group']], hue='Cluster_Group')

Pair Plot showing the cluster groups with the type of contact 

In [ ]:
sns.pairplot(data_temp[['Is_Electronic_check','PaperlessBilling_Yes','Cluster_Group']], hue='Cluster_Group')

Pair Plot showing the cluster groups with the type of payment method

In [ ]:
subgroup = data_temp[['Is_Contract_Month','Contract_One year','Contract_Two year','Is_Electronic_check','PaperlessBilling_Yes','Cluster_Group']]

In [ ]:
cluster_data = subgroup.groupby('Cluster_Group')

In [ ]:
cluster_data.plot(subplots=True,)

In [ ]:
# Print the cluster centroids
print(km.cluster_centroids_)

**Decision Tree**

According to chapter three of a research paper done at The Princeton university, Decision tree learning is a supervised learning strategy for approximating discrete-valued target functions in which this machine learning is represented by a graphical decision tree. Learned trees can and is typically represented as a set of if-then-else rules to improve human readability and experience. The goal of a decision tree is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. Unfortunately, the deeper the tree, the more complex the rules and fitter the model. If a decision tree displays overfitting then, it should be noted that it becomes difficult to predict a target value. 


In [ ]:
X_data = data_temp2[['SeniorCitizen','tenure','MonthlyCharges','TotalCharges','Is_Electronic_check','Is_Contract_Month','StreamingTV_Yes']]
y_data = data_temp2['Churn_Yes']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
clf = DecisionTreeClassifier(max_depth=2, criterion='entropy')

In [ ]:
clf.fit(X_data, y_data)

In [ ]:
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [ ]:
y_data

In [ ]:
pd.DataFrame([ "%.2f%%" % perc for perc in (clf.feature_importances_ * 100) ], index = X_data.columns, columns = ['Feature Significance in Decision Tree'])

In [ ]:
import graphviz

In [ ]:
dot_data = tree.export_graphviz(clf,out_file=None, 
                                feature_names=X_data.columns,
                                class_names = ['No', 'Yes'],
                         filled=True, rounded=True,  proportion=True,
                                node_ids=True, #impurity=False,
                         special_characters=True)

**Ploting the Decision**

In [ ]:
graph = graphviz.Source(dot_data) 
graph

**Justification for use of algorithms with data set**

Decision Tree
* The decision tree algorithm was a good fit for this data set as it is a supervised learning algorithm that outlines an outcome based on a series of related conditions based on the features of the data set. In this case ‘Churn’ was the target output and the goal was to find out what was making customers churn or leave the company. The decision tree gives a graphical view to show the features that are affecting churn and how they affect it. This gives a clear idea of what is making customers churn and what can be done to prevent churning.


K-Modes
* The K-Modes algorithm was used to see what groups of clusters of customers was churning the most and to see what were the common values for the different features among them. This is very valuable as an effort can be made to analyze what is increasing the likeliness of the a customer churning and rectify the problem. Also we can see what cluster has the lowest churn rate and try to increase those conditions in order to increase customer satisfaction as that cluster would be the most satisfied.


Logistic Regression
* Logistic Regression was used because it is used to predict a target variable and it is used for classification data. It also works well with large sample sizes and binary input. We can use it to predict what would cause customers to churn based on the values of the various features and how much they impact the churn rate. 


**Description of how algorithms operated on data**

**Logistic Regression**
* For us to have performed the logistic regression, we had to divide our columns in two distinctions ; dependent variable (or target variable) and independent variable(or feature variables). As mentioned previously, our target variable is ‘Churn’. The data is split in a train set, which trains the model, and a test set. The train_test_set() function was used to achieve this, and a 70/30 ratio used. This means 70% of our data was used to train the model, while the remaining 30% was used as the test set. In order to develop our model, we first import the Logistic Regression module and create a logistic regression classifier object, using the LogisticRegression() function. Our object was called ‘model’. We then fit our model on the train set using the fit() function and predict the test test using the predict() function. The output which is then produced is a confusion matrix, using a heat map. The visualizes the number of correct and incorrect predicts made by our model. Based on our matrix, our model made a correct prediction of 1669 and 441 incorrect predictions. Our model correctly predicted 80% of the data.


**K-Modes**
* The Kmodes algorithm forms clusters based on the number of matching categories between data points. The Kmodes() function is passed four parameters; the amount of clusters we would like to form, initialisation type, number of iterations which will be run with different centroid seeds and the verbose. We then store the results in a variable ‘km’.The k-modes approach  uses a dissimilarity measure, using modes to represent cluster centers and updating modes with the most frequent categorical values in each iteration of the clustering process. At the end of the iterations, the best run is selected. The fit_predict function accepts our dataset and forms the clusters, which we store in a variable called ‘clusters’. We use a bar graph to illustrate the output of the different cluster groups.


**Decision Tree**
* A decision tree was used to create a model which predicts the target variable by learning simple decision rules based on the dependent variables. The X_data consists of SeniorCitizen, Tenure,MonthlyCharges, TotalCharges, Is_Electronic_check, Is_Contract_Month, StreamingTV_Yes and the Y_data is ‘Churn_Yes’.  We import the tree and DecisionTreeClassifier() modules to help us build the tree. The DecisionTreeClassifier() takes a number of parameters. A few of these parameters include criterion which defines the quality of a split, max_features, which defines the number of features to consider when looking for the best split, max_depth, which states the maximum depth of the tree, max_leaf_nodes which defines the maximum number of possible leaf nodes. We use the fit() function to accept the X_data and Y_data,in order  to train the tree. The tree.export_graphviz() function accepts parameters which defines the placement of the data on the tree, specifications and headings. That information is then stored in our dot_data variable, which is then passed to the graphviz.Source() and stored in a variable ‘graph’, which is used to produce the tree visualization.



**Outline of methods/activities done at each step of the KDD process**

* The term Knowledge Discovery in Databases, or KDD for short, refers to the broad process of finding knowledge in data, and emphasizes the "high-level" application of particular data mining methods. The unifying goal of the KDD process is to extract knowledge from data in the context of large databases. (Piatetsky-Shapiro, 1999). Additionally, KDD is an iterative process where evaluation measures can be enhanced, mining can be refined, new data can be integrated and transformed in order to get different and more appropriate results. 
* Selection Process
Data selection in the KDD model can defined as the process where data relevant to the analysis is decided and retrieved from the data collection (complete dataset). During this process, based on the type of algorithm that was implemented, specific columns were included or removed from the main dataset. Specific algorithms required only categorical data while other times, only numerical data was needed to perform the analysis. 
Decision trees required only numerical data that had a significance on the target variable therefore, only specific columns were included. 
Regression required only discrete-valued data.
K-modes required only categorical data.  
* Preprocessing
Preprocessing is a data mining technique which essentially, prepares data before it is fed into an algorithm. When data is sourced, it may have several bad or inconsistent entries which will adversely impact the outcome of the algorithm. Data must first be cleaned up to ensure we achieve the best results. For our preprocessing, we ensured that there are no null values or bad data before conducting our algorithms. Also, we transformed the ‘Total_Charges’ column to float, as it was being recognized as an object, which would hinder us from doing any form of operations to it. 
* Transformation
Data Transformation is defined as the process of transforming data into appropriate form required by mining procedure. Data may need to be changed into different forms in order to work properly with certain algorithms. There are two main types of variable, categorical and numerical. Categorical data has sub-types: dichotomous, discrete, nominal and may be ranked. Numerical data is continuous. Firstly, all Yes/No variables were transformed to 1/0 as it better suited the algorithms selected. 
 The dataset had mostly categorical data and the data was transformed to ensure that the dataset had uniform discrete data. We then dropped the the variables that according to our analysis were unimportant these were, : 'StreamingTV_No internet service', 'StreamingMovies_No internet service', ‘gender’, ‘class’, ‘customerID’, 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport' and 'PaymentMethod'.
For the K-Modes algorithm the columns 'TotalCharges', 'tenure' and 'MonthlyCharges' were dropped because these were continuous numerical data and K-Modes clustering deals with only categorical data. 
No transformation was required for the decision tree as the data was already in a suitable form.
* Data Mining
Through reading and understanding, data mining is the process of using clever techniques for analyzing and extracting useful  hidden patterns of data according to different perspectives for categorization into useful information. Our dataset consisted of both categorical and numerical data. Additionally, it also consisted of a target variable (Churn rate) therefore we had the opportunity to utilize both supervised and unsupervised machine learning algorithms to analyze and extract patterns and also make predictions using dependent and independent variables. Supervised learning is typically done in the context of classification, when we want to map input to output labels, or regression, when we want to map input to a continuous output. We utilized two (2) supervised algorithm, namely logistic regression and Decision tree. On the other hand, the most common tasks within unsupervised learning are clustering and we used just one algorithm:K-Modes to display this. 
* Supervised Learning
The decision tree algorithm used in our project firstly required the several of the dataset columns to be transformed into  numerical data. This was achieved using the python function called lambda. This newly transformed data now was able to be placed in a decision tree which allowed us to analyze the if-then-else structure to determine the sequence/reasons that might affect a customer’s decision to churn. 
The logistic regression used in our project was implemented because we as a group of aspiring data scientists decided it was important to know if we could possibly use the existing data by identifying patterns and extracting knowledge in order to make future predictions. Again, the data was transformed and the newly transformed dataset was used to train and test and produced a confusion matrix and regression graph which allowed us to make interpretations and evaluations. 
* Unsupervised Learning
During this stage of the KDD process, clustering categorical data is an important research technique in data mining in order to provide insights, understand and provide evaluations and interpretations. This algorithm was used to see what groups of clusters of customers was churning the most and to see what were the common values for the different features among them. 
Interpretation/Evaluation
Based on the logistic regression algorithm, we were able to create a confusion matrix and the results from the confusion matrix told us that 1404 and 265 are the number of correct predictions. 151 and 290 are the number of incorrect predictions.Consequently, we had a classification rate of 80%, and we considered that  as a good accuracy because we could predict future trends based on the patterns we identified in this dataset. 
Additionally, the decision tree provided insights on the reasons why a customer may decide to churn. We saw that using the if-then-else structure, if a customer’s tenure is less than or is equal to 17.5 AND their monthly charges are greater than approximately 69 dollars then they will definitely churn. In summary, the algorithms with their specific functions helped to answer the aim of the project which was to determine the reasons why a customer may churn. 
